# Custom Object Detection - Low Clearance Sign Detector

In [0]:
# clone low-clearance-detector repo
!git clone https://github.com/lbborkowski/low-clearance-detector.git

## Setup

### Install

Install tensorflow 1.* -- tensorflow object detection api is not compatible with tensorflow 2.*

In [0]:
pip install tensorflow-gpu==1.*

Downgrade to numpy 1.17.4 to avoid an error during evaluation

Requires a restart of the runtime before continuing through the notebook - click **RESTART RUNTIME** in output below

In [0]:
!pip install numpy==1.17.4

Make sure you have `pycocotools` installed

In [0]:
!pip install pycocotools

Get `tensorflow/models` or `cd` to parent directory of the repository

In [0]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Compile protobufs and install the object_detection package

In [0]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [0]:
%%bash 
cd models/research
pip install .

Add necessary libraries to PYTHONPATH

In [0]:
%set_env PYTHONPATH=$PYTHONPATH:/content/models/research:/content/models/research/slim

### Imports

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

Import the object detection module

In [0]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [0]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

## Training

### Create TFRecords for train and test images

In [0]:
%cd /content/low-clearance-detector/

In [0]:
# Create train TFRecord
!python generate_tfrecord.py --label='low-clearance-sign' --csv_input=/content/low-clearance-detector/annotations/train_labels.csv --img_path=/content/low-clearance-detector/images/train --output_path=/content/low-clearance-detector/annotations/train.record

In [0]:
# Create test TFRecord
!python generate_tfrecord.py --label='low-clearance-sign' --csv_input=/content/low-clearance-detector/annotations/test_labels.csv --img_path=/content/low-clearance-detector/images/test --output_path=/content/low-clearance-detector/annotations/test.record

### Train custom object detection model
Train the model for 6500 steps

In [0]:
%cd /content/low-clearance-detector
!python model_main.py --pipeline_config_path=training/faster_rcnn_inception_v2_coco_lcsd.config --model_dir=training/ --num_train_steps=6500 --alsologtostderr

#### Setup and run tensorboard
Visualize training losses and evaluation results

In [0]:
# Uninstall tensorboard plugin to avoid conflict
pip uninstall -y tensorboard-plugin-wit

In [0]:
from tensorflow import summary
%load_ext tensorboard

In [0]:
%tensorboard --logdir training/

### Export trained low clearance sign detection model

In [0]:
%cd /content/low-clearance-detector

In [0]:
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=training/faster_rcnn_inception_v2_coco_lcsd.config \
    --output_directory=trained-model \
    --trained_checkpoint_prefix=training/model.ckpt-6500

## Inference

### Imports

#### Object detection imports
Imports from the object detection module

In [0]:
%cd /content/models/research/object_detection

In [0]:
from utils import label_map_util

from utils import visualization_utils as vis_util

### Model preparation 

#### Variables

Any model exported using the "export_inference_graph.py" tool can be loaded here simply by changing "PATH_TO_FROZEN_GRAPH" to point to a new .pb file

In [0]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = '/content/low-clearance-detector/trained-model/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/content/low-clearance-detector/annotations/label_map.pbtxt'

#### Load a (frozen) tensorflow model into memory

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

#### Loading label map

In [0]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
print(category_index)

#### Helper code

In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

#### Inference function
Perform inference one image at a time

In [0]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

### Detection - validation images
Detect low clearance signs in validation image set

In [0]:
import os
import glob

PATH_TO_TEST_IMAGES_DIR = '/content/low-clearance-detector/images/valid'
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))
print(TEST_IMAGE_PATHS)

# Size, in inches, of the output images
IMAGE_SIZE = (16, 16)

#### Perform inference on all validation images
Loop over validation set and output images containing detected low clearance signs with its location denoted using a bounding box

In [0]:
%matplotlib inline
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.axis('off')
  plt.imshow(image_np)
  print(output_dict['detection_scores'][0], output_dict['detection_scores'][1])